In [2]:
import numpy as np
import os
import cv2
from shared_fonctions import *
import matplotlib.pyplot as plt

#new_classes_importance = [1,2,4,5,0,3,6,7] #camvid
#filtre_attribution_label = [0.97,0.4,0.7,0.95,0.5,0.8,0.99,0.99]
#label_assignation = [6,3,3,0,0,5,4,3,2,1,2,7,7]

classes_names = ['void','ground','construction','object','nature','sky','human','vehicle']
ctscp_classes_importance = [6,7,3,2,4,5,1,0]
ctscp_filtre_attribution_label = [0.5,0.8,0.8,0.4,0.7,0.5,0.2,0.75]
# ce filtre est bien sur zurich !!
ctscp_proba_keep = [1,0.25,0.4,1,0.4,1,1,1]
proportions_max = 0.25

nb_classes=8
data_shape = 240*480

width=480
height=240

window_size=(64,64) #format width*height
pad=21
nb_classes = 8

def get_camvid_lists():
    with open(os.getcwd()+'/CamVid/train.txt') as f:
        train_img_list = f.readlines()
        train_img_list = [line.split(' ') for line in train_img_list]

    with open(os.getcwd()+'/CamVid/test.txt') as f:
        test_img_list = f.readlines()
        test_img_list = [line.split(' ') for line in test_img_list]
    return train_img_list, test_img_list

def load_camvid(train_or_test,img_or_label,num):
    if(train_or_test=='train' and img_or_label=='label'):
        return cv2.imread(os.getcwd() + train_img_list[num][1][7:][:-1])[:,:,0]
    elif(train_or_test=='test' and img_or_label=='label'):
        return cv2.imread(os.getcwd() + test_img_list[num][1][7:][:-1])[:,:,0]
    elif(train_or_test=='train' and img_or_label=='img'):
        return cv2.imread(os.getcwd() + train_img_list[num][0][7:])
    elif(train_or_test=='test' and img_or_label=='img'):
        return cv2.imread(os.getcwd() + test_img_list[num][0][7:])
       
def one_hot(n):
    a=np.zeros(nb_classes)
    a[n]=1
    return a

def attribuer_label_centre(labels):
    lab = labels[12:43,12:43]
    classes = [label_assignation[int(q)] for q in lab.flatten()]
    hist = np.bincount(classes,minlength = nb_classes)
    hist = 1.0*hist/np.sum(hist)
    #main_class = np.argmax(hist)
    for i in new_classes_importance:
        if(hist[i]>filtre_attribution_label[i]):
            if(np.random.rand()<ctscp_filtre_attribution_label[i]):
                return i
            else:
                return -1
    return -1 # si ce n'est pas un bon exemple.
    
def attribuer_label_vote(labels, p):
    cl = [int(q) for q in labels.flatten()]
    cl = np.bincount(cl)
    cl = [cl[i]*np.sqrt(class_weighting[i]) for i in range(len(cl))]
    cl= cl/np.sum(cl) # vecteur de probabilité de chaque classe ! et oué
    main_class = np.argmax(cl) # probabilité d'appartenance à la classe dominante
    if(cl[main_class]>p):
        return main_class
    else:
        return -1

def prep_data(nb_images,path,kind):
    print("Buiding dataset...")
    goodExample = 0
    go_file = 0
    train_data = []
    train_label = []
    for i in range(nb_images):
        print "Image n°",i
        labels = load_camvid(kind,'label',i)
        img = load_camvid(kind,'img',i)
        for j in range(0,width-window_size[0],pad):
            for k in range(0,height-window_size[1],pad):
                classe = attribuer_label_centre(labels[k:k+window_size[1],j:j+window_size[0]])
                if (classe!=-1):
                    goodExample = goodExample + 1
                    go_file +=1
                    window = img[k:k+window_size[1],j:j+window_size[0]] #fenetre glissante
                    train_data.append(normalized(window))
                    train_label.append(one_hot(classe))
        print(goodExample)
        if(go_file>10000 or i==(nb_images-1)):
            go_file = 0
            print goodExample/10000
            np.save('data/'+path+'/'+kind+'_data_'+str(goodExample/10000)+'.npy',train_data)
            np.save('data/'+path+'/'+kind+'_label_'+str(goodExample/10000)+'.npy',train_label)
            del train_label
            train_label = []
            del train_data
            train_data = []
    print("Dataset built")
    print("Good examples =", goodExample)
    print("Classes :",new_classes_names)
    print("Histogramme", histoclass(train_data))

In [3]:
train_img_list, test_img_list = get_camvid_lists()

nb_test_examples = len(test_img_list)
nb_train_examples = len(train_img_list)
#prep_data(nb_train_examples,'camvid','train')
#prep_data(nb_test_examples,'camvid','test')
print(ctscp_proba_keep)

[1, 0.25, 0.4, 1, 0.4, 1, 1, 1]


In [4]:
train_data,train_label = load_camvid_train_data([1])
print histoclass(train_label)

[2664   93 1123 2339  126 1746 2043   28]


In [5]:
from cityscapes_labels import *

def get_cities(kind): # kind = 'train','test' or 'val'
    return os.listdir('cityscape/labels/'+kind);

def get_cityscape_category(ll):
    id_to_category = { label.id : label.categoryId for label in labels}
    return [id_to_category[i] for i in ll.flatten()]


def get_city_data(kind,city): # kind = 'train','test' or 'val'
    print "getting ",city
    label_path = 'small_cityscape/labels/'+kind+'/'+city+'/'
    img_path = 'small_cityscape/images/'+kind+'/'+city+'/'
    img_files_list = os.listdir(label_path)
    img_set = []
    label_set = []
    for i in range(len(img_files_list)):
        if i%30==0:
            print i
        img_set.append(cv2.imread(img_path+str(i)+'.png'))
        label_set.append(cv2.imread(label_path+str(i)+'.png'))
    return img_set,label_set


#img_set,label_set = get_city_data('train','stuttgart')

#cv2.imshow(img_set[1])

In [6]:
def ctscp_histo(lab):
    classes = get_cityscape_category(lab)
    hist = np.bincount(classes,minlength = nb_classes)
    return hist

def ctscp_categorize(hist,filtre,proba):
    hist = 1.0*hist/np.sum(hist)
    #main_class = np.argmax(hist)
    for i in ctscp_classes_importance:
        if(hist[i]>filtre[i]):
            if(np.random.rand()<proba[i]):
                return i
            else:
                return -1    
    return -1 # si ce n'est pas un bon exemple.

In [7]:
def prep_cityscape_data(cities,kind,phase): # kind = train,test, val ; phase = test or store
    img_set,label_set = [],[]
    print("Buiding dataset...")
    for city in cities:
        train_data = [[],[],[],[],[],[],[]]
        train_label = [[],[],[],[],[],[],[]]
        histo = []
        params = []
        del img_set,label_set
        img_set,label_set = get_city_data(kind,city)
        for i in range(len(img_set)):
            print '-',
            compteur = 0
            goodExample = 0
            labels = label_set[i]
            img = img_set[i]
            for j in range(0,width-window_size[0],pad):
                for k in range(0,height-window_size[1],pad):
                    hist = ctscp_histo(labels[k+12:k+window_size[1]-12,j+12:j+window_size[0]-12])
                    #classe = ctscp_categorize(hist,ctscp_filtre_attribution_label)
                    compteur+=1
                    params.append((i,j,k))
                    histo.append(hist)
                    # if (classe!=-1):
                #    goodExample+=1
                 #       window = img[k:k+window_size[1],j:j+window_size[0]] #fenetre glissante
                #      if(phase=='store'):
                   #         train_data[.append(normalized(window))
                    #        train_label.append(one_hot(classe))
            print goodExample,
        if(phase=='store'):
            np.save('data/cityscape/'+kind+'_data_'+city+'.npy',train_data)
            np.save('data/cityscape/'+kind+'_label_'+city+'.npy',train_label)
        np.save('data/cityscape/'+kind+'/histo_'+city+'.npy',histo)
        np.save('data/cityscape/'+kind+'/histo_'+city+'.npy',histo)
    print("Dataset built")
    print("Good examples =", goodExample)
    print("Classes :",classes_names)
    print("Histogramme", histoclass(train_label))

def read_histo(kind, city):
    return np.load('data/cityscape/'+kind+'_histo_'+city+'.npy')

def get_repartition_from_histo(histo,filtre):
    res = np.zeros(nb_classes)
    for h in histo:
        classe = ctscp_categorize(h, filtre)
        if(classe!=-1):
            res[classe]+=1
    return res
            
    

In [8]:
#prep_cityscape_data(['zurich'],'train','test') # a effectuer avant de partir manger

Buiding dataset...
getting  zurich
0
30
60
90
120
- 0 - 0 - 0 -

KeyboardInterrupt: 

In [ ]:
histo = read_histo('train','zurich')
a= get_repartition_from_histo(histo,[0.5,0.8,0.8,0.4,0.7,0.5,0.2,0.75])
print a.sum()


In [ ]:
def reequilibrage(train_data,train_label):
    perm = np.random.permutation
    
    n = len(train_data)
    for i in range(nb_classes):
        nb_i = len(train_data[train_label==i])
        if nb_i>0.25*n:
            

In [ ]:
train_data,train_label = load_ctscp_data('cityscape','train',['zurich'])

In [ ]:
show_sample(train_data,train_label,0,classes_names)

In [ ]:
kind = 'train'
city = 'strasbourg'
label_path = 'cityscape/labels/'+kind+'/'+city+'/'
img_path = 'cityscape/images/'+kind+'/'+city+'/'
label_list = os.listdir(label_path)
label_list = [f for f in label_list if 'labelIds' in f]
img_files_list = os.listdir(img_path)
label_list.sort()
img_files_list.sort()
print len(label_list)
print len(img_files_list)

In [11]:

def smaller_train(cities):
    for city in cities:
        old_label_path = 'cityscape/labels/test/'+city+'/'
        old_img_path = 'cityscape/images/test/'+city+'/'
        label_list = os.listdir(old_label_path)
        img_files_list = os.listdir(old_img_path)
        label_list = [f for f in label_list if 'labelIds' in f]
        img_files_list.sort()
        label_list.sort()
        for i in range(len(img_files_list)):
            if i%30==0:
                print i
            img = cv2.resize(cv2.imread(old_img_path+img_files_list[i]),(480,240))
            
            cv2.imwrite('small_cityscape/images/test/'+city+'/'+str(i)+'.png',img)
            del img
            lbl = cv2.resize(cv2.imread(old_label_path+label_list[i]),(480,240))
            lbl = cat(lbl)
            cv2.imwrite('small_cityscape/labels/test/'+city+'/'+str(i)+'.png',lbl)
            print '.',
        print city," ok"
    

In [37]:
def org_label(cities,kind):
    for city in cities:
        old_label_path = 'cityscape/labels/'+kind+'/'+city+'/'
        label_list = os.listdir(old_label_path)
        label_list = [f for f in label_list if 'labelIds' in f]
        label_list.sort()
        for i in range(len(label_list)):
            if i%30==0:
                print i
            print old_label_path+label_list[i]
            lbl = cv2.resize(cv2.imread(old_label_path+label_list[i]),(480,240))
            cv2.imwrite('small_cityscape/30_classes_labels/'+kind+'/'+city+'/'+str(i)+'.png',lbl)
            print '.',
        print city," ok"

In [38]:
org_label(get_cities('test'),'test')
org_label(get_cities('train'),'train')


0
cityscape/labels/test/bonn/bonn_000000_000019_gtFine_labelIds.png
. cityscape/labels/test/bonn/bonn_000001_000019_gtFine_labelIds.png
. cityscape/labels/test/bonn/bonn_000002_000019_gtFine_labelIds.png
. cityscape/labels/test/bonn/bonn_000003_000019_gtFine_labelIds.png
. cityscape/labels/test/bonn/bonn_000004_000019_gtFine_labelIds.png
. cityscape/labels/test/bonn/bonn_000005_000019_gtFine_labelIds.png
. cityscape/labels/test/bonn/bonn_000006_000019_gtFine_labelIds.png
. cityscape/labels/test/bonn/bonn_000007_000019_gtFine_labelIds.png
. cityscape/labels/test/bonn/bonn_000008_000019_gtFine_labelIds.png
. cityscape/labels/test/bonn/bonn_000009_000019_gtFine_labelIds.png
. cityscape/labels/test/bonn/bonn_000010_000019_gtFine_labelIds.png
. cityscape/labels/test/bonn/bonn_000011_000019_gtFine_labelIds.png
. cityscape/labels/test/bonn/bonn_000012_000019_gtFine_labelIds.png
. cityscape/labels/test/bonn/bonn_000013_000019_gtFine_labelIds.png
. cityscape/labels/test/bonn/bonn_000014_000019_

KeyboardInterrupt: 

In [50]:
a = cv2.imread('cityscape/labels/test/berlin/berlin_000069_000019_gtFine_labelIds.png')


In [51]:
print np.bincount(a.flatten(),minlength = 35)


[5951952  244602       0   94902       0       0       0       0       0
       0       0       0       0       0       0       0       0       0
       0       0       0       0       0       0       0       0       0
       0       0       0       0       0       0       0       0]


cities = get_cities('test')
cities2 = get_cities('train')
for city in cities:
    os.makedirs('small_cityscape/30classes_labels/test/'+city)
for city in cities2:
    os.makedirs('small_cityscape/30classes_labels/train/'+city) 

In [17]:
cities
smaller_train(cities[3:]) #a relancer sans bonn berlin mainz

0
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 30
. . . . . . . . . . . . . . . . . . . . . . . . . . . . leverkusen  ok
0
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 30
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 60
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 90
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 120
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 150
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 180
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 210
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 240
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 270
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 300
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 330
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 360
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 390
. . . . . . . . munich  ok
0
. . 

In [14]:
id_to_category = { label.id : label.categoryId for label in labels}

def cat(arr):
    (a,b,c) = arr.shape
    res = np.empty_like(arr)
    for i in range(a):
        for j in range(b):
            for k in range(3):
                res[i,j,k] = id_to_category[arr[i,j,0]]
    return res

In [ ]:
cities = get_cities('train')

In [ ]:
def prep_cityscape_histo(cities,kind): # kind = train,test, val ; phase = test or store
    path = 'small_cityscape/histo'
    print("Buiding dataset...")
    for city in cities:
        print "getting ",city
        histo = []
        params = []
        img_set,label_set = get_city_data(kind,city)
        for i in range(len(img_set)):
            print '-',
            compteur = 0
            goodExample = 0
            labels = label_set[i]
            img = img_set[i]
            for j in range(0,width-window_size[0],pad):
                for k in range(0,height-window_size[1],pad):
                    hist = np.bincount(labels[k+12:k+window_size[1]-12,j+12:j+window_size[0]-12].flatten(),minlength = nb_classes)
                    #classe = ctscp_categorize(hist,ctscp_filtre_attribution_label)
                    compteur+=1
                    params.append((i,j,k))
                    histo.append(hist)
                    # if (classe!=-1):
                #    goodExample+=1
                 #       window = img[k:k+window_size[1],j:j+window_size[0]] #fenetre glissante
                #      if(phase=='store'):
                   #         train_data[.append(normalized(window))
                    #        train_label.append(one_hot(classe))
            print goodExample,
        np.save('small_cityscape/histo/'+kind+'/'+city+'_histo.npy',histo)
        np.save('small_cityscape/histo/'+kind+'/'+city+'_params.npy',params)
    print("Dataset built")
    print("Good examples =", goodExample)
    print("Classes :",classes_names)
    print("Histogramme", histo[1])

def read_histo(kind, city):
    return np.load('data/cityscape/'+kind+'_histo_'+city+'.npy')

def get_repartition_from_histo(histo,filtre,proba):
    res = np.zeros(nb_classes)
    for h in histo:
        classe = ctscp_categorize(h, filtre,proba)
        if(classe!=-1):
            res[classe]+=1
    return res

In [ ]:
cities = get_cities('test')
#cities = ['zurich']
prep_cityscape_histo(cities,'test')


In [ ]:
params = np.load('small_cityscape/histo/train/erfurt_histo.npy')
params = np.load('small_cityscape/histo/train/erfurt_params.npy')


In [ ]:
params

In [ ]:
def create_ctscp_dataset(cities):
    img_set = [[],[],[],[],[],[],[],[]]
    for city in cities:
        histo = np.load('small_cityscape/histo/train/'+city+'_histo.npy')
        params = np.load('small_cityscape/histo/train/'+city+'_params.npy')
        print "getting ",city
        current_i = -1
        for count in range(len(params)):
            goodExample = 0
            h = histo[count]
            (i,j,k) = params[count]
            if i!=current_i:
                current_i = i
                img = cv2.imread('small_cityscape/images/train/'+city+'/'+str(i)+'.png')
            classe = ctscp_categorize(h,ctscp_filtre_attribution_label, ctscp_proba_keep)
            if(classe!=-1):
                goodExample+=1
                img 
                window = img[k:k+window_size[1],j:j+window_size[0]]
                img_set[classe].append(normalized(window))
        print goodExample,"-",
    for q in range(nb_classes):
        np.save('data/cityscape/train/'+str(q)+'/all_cities.npy',img_set[q])
        
                
                
            
        
    

In [ ]:
create_ctscp_dataset(get_cities('train'))

In [2]:
def load_class(i, nb_max,cities = None):
    a=[]
    if(cities):
        for city in cities:
            a.extend(np.load('data/cityscape/train/'+str(i)+'/'+city+'.npy'))
    else:
        a = np.load('data/cityscape/train/'+str(i)+'/all_cities.npy')
    np.random.shuffle(a)
    return a[:nb_max]


In [3]:
def create_label_matrix(i,nb):
    res = np.zeros((nb,nb_classes))
    res[:,i]=np.ones(nb)
    return res

In [6]:

for i in range(nb_classes):
    print i
    arr = load_class(i,1000)
    np.save('data/cityscape/train/'+str(i)+'/random_1000.npy',arr)

0
1
2
3
4
5
6
7


In [ ]:
res = create_label_matrix(1,3)
print res




In [ ]:
time



In [ ]:
time.time()